In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz

In [2]:
import os
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [3]:
!pip install findspark

In [4]:
import findspark
findspark.init()

In [5]:
!pip3 install pyspark==3.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.7/204.7 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 23.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044159 sha256=d4c9244ff70a3cb5682f67b094ba0f75fd7186298aab496da1f10a7555ca32d6
  Stored in directory: /root/.cache/pip/wheels/b1/bb/8b/ca24d3f756f2ed967225b0871898869db676eb5846df5adc56
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


<h1><center>Инициализация</center></h1>

In [11]:
from pyspark import SparkContext, SparkConf
import pyspark.sql as sql
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import udf, col, max, sum, countDistinct

In [7]:
spark = SparkSession \
    .builder \
    .appName("L1_interactive_bike_analysis") \
    .getOrCreate()

<h1><center>Загрузка данных</center></h1>

In [8]:
import os
data_path = os.path.join(os.curdir, "data")
trips_path = os.path.join("trips.csv")
stations_path = os.path.join("stations.csv")

In [9]:
trips = spark.read \
        .option("header", True) \
        .option("inferSchema", True) \
        .option("DateTimeFormat", 'M/d/y H:m') \
        .csv(trips_path)

stat = spark.read \
        .option("header", True) \
        .option("inferSchema", True) \
        .option("DateTimeFormat", 'M/d/y H:m') \
        .csv(stations_path)

<a id='Задачи для данных велопарковок Сан-Франциско (trips.csv, stations.csv):'></a>
## Задачи для данных велопарковок Сан-Франциско (trips.csv, stations.csv):
><li>Найти велосипед с максимальным временем пробега.
><li>Найти наибольшее геодезическое расстояние между станциями.
><li>Найти путь велосипеда с максимальным временем пробега через станции.
><li>Найти количество велосипедов в системе.
><li>Найти пользователей потративших на поездки более 3 часов.

<h1><center>Найти велосипед с максимальным временем пробега</center></h1>

In [22]:
# Группируем велосипеды по id и считаем время пробега для каждого велосипеда
trip_duration = trips.groupBy("bike_id").agg(sum(col("duration")).alias("duration"))
# Выбираем велосипед с максимальным пробегом
trips_duration = trip_duration.orderBy(col("duration").desc()).first()
bike_id = trips_duration["bike_id"]
duration = trips_duration["duration"]
print(f"Велосипед {bike_id} с максимальным временем пробега {duration}")

Велосипед 535 с максимальным временем пробега 18611693


<h1><center>Найти наибольшее геодезическое расстояние между станциями</center></h1>

In [33]:
from math import sqrt, radians, sin, cos, atan2

def distance(lat1, long1, lat2, long2):
    R = 6373 # Радиус Земли
    lat1 = radians(lat1)
    lat2 = radians(lat2)
    long1 = radians(long1)
    long2 = radians(long2)
    # Вычисляем геодезическое расстояние
    distance = R * (2 * atan2(sqrt(sin((lat2 - lat1) / 2)**2 + cos(lat1) * cos(lat2) * sin((long2 - long1) / 2)**2), sqrt(1 - (sin((lat2 - lat1) / 2)**2 + cos(lat1) * cos(lat2) * sin((long2 - long1) / 2)**2))))
    return distance
# Функция в pyspark.sql.functions.udf
g_dist = udf(distance, DoubleType())
# Получаем все возможные пары станций
station = stat.alias("station1").crossJoin(stat.alias("station2"))
# Вычисляем расстояние для каждой пары станций
station_distance = station.withColumn("geodesic_distance", g_dist(col("station1.lat"), col("station1.long"), col("station2.lat"), col("station2.long")))
# Максимальное геодезическое расстояние для каждой пары станций
dist = station_distance.selectExpr("max(geodesic_distance) as max_distance").collect()[0]["max_distance"]
print(f"Наибольшее геодезическое расстояние между станциями {dist}")

Наибольшее геодезическое расстояние между станциями 69.9428256877473


<h1>Найти путь велосипеда с максимальным временем пробега через станции</h1>

In [34]:
# Сортируем и выбираем наиболее длительную поездку
duration = trips.select("start_station_name", "end_station_name", "duration").orderBy(col("duration").desc()).first()
# Получаем начальную станцию
first = duration["start_station_name"]
# Получаем конечную станцию
second = duration["end_station_name"]
# Получаем время поездки
all_time = duration["duration"]
print(f"Самая длинная поездка из \"{first}\" в \"{second}\" заняла {all_time} секунд")

Самая длинная поездка из "South Van Ness at Market" в "2nd at Folsom" заняла 17270400 секунд


<h1><center>Найти количество велосипедов в системе</center></h1>

In [35]:
# Группировка по id велосипеда и подсчет уникальных значений id
count = trips.agg(countDistinct("bike_id").alias("bike_count")).collect()[0]["bike_count"]
print(f"Количество велосипедов в системе: {count}")

Количество велосипедов в системе: 700


<h1><center>Найти пользователей потративших на поездки более 3 часов</center></h1>

In [37]:
# Группируем велосипеды по id и считаем общее время поездки
users = trips.groupBy("bike_id").sum("duration").withColumnRenamed("sum(duration)", "time")
users.filter("time>10800").show()

+-------+-------+
|bike_id|   time|
+-------+-------+
|    471|1718831|
|    496|1679568|
|    148| 332138|
|    463|1722796|
|    540|1752835|
|    392|1789476|
|    623|2037219|
|    243| 307458|
|    516|1896751|
|     31| 407907|
|    580|1034382|
|    137|1529200|
|    251|1282980|
|    451|1695574|
|     85|1214769|
|    458|1647080|
|     65| 216922|
|    588| 266415|
|    255| 396395|
|     53| 226389|
+-------+-------+
only showing top 20 rows

